In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [15]:
locB_a1_temps = pd.read_excel("Country B.xlsx", sheet_name=2, skiprows=0).dropna().reset_index(drop=True)
locB_a1_temps['Date'] = datetime(1970,1,1)
locB_a1_temps['Avg_Temp'] = np.nan
for i, row in locB_a1_temps.iterrows():
    yr, mt, d, maxt, mint, dt, avgt = row.values
    locB_a1_temps.loc[i, 'Date'] = datetime(int(yr), int(mt), int(d))
    try:
        locB_a1_temps.loc[i, 'Avg_Temp'] = (float(maxt) + float(mint))/2.0
    except:
        continue
locB_a1_temps['doy'] = locB_a1_temps.Date.apply(lambda x: x.day_of_year)
locB_a1_temps = locB_a1_temps.rename(columns={'YEAR':'year', "Tmax (deg C)":'Max_Temp', "Tmin (deg C)":'Min_Temp', "MONTH":"Month"}).drop("DAY",axis=1)
locB_a1_temps['Max_Temp'] = locB_a1_temps.Max_Temp.apply(lambda x: float(x) if x not in ['n.a.'] else np.nan)
locB_a1_temps

,year,Month,Max_Temp,Min_Temp,Date,Avg_Temp,doy
0,1986,1,29.8,23.5,1986-01-01,26.65,1
1,1986,1,29.6,23.1,1986-01-02,26.35,2
2,1986,1,29.4,24.1,1986-01-03,26.75,3
3,1986,1,29.4,22.0,1986-01-04,25.70,4
4,1986,1,29.5,22.4,1986-01-05,25.95,5
...,...,...,...,...,...,...,...
13119,2021,12,30.6,24.0,2021-12-27,27.30,361
13120,2021,12,30.4,23.0,2021-12-28,26.70,362
13121,2021,12,30.9,24.4,2021-12-29,27.65,363
13122,2021,12,30.6,25.4,2021-12-30,28.00,364


In [21]:
locB_a1_rain = pd.read_excel("Country B.xlsx", sheet_name=1, skiprows=0).dropna().reset_index(drop=True)
locB_a1_rain['Date'] = datetime(1970,1,1)
for i, row in locB_a1_rain.iterrows():
    yr, mt, d, rain, dt = row.values
    locB_a1_rain.loc[i, 'Date'] = datetime(int(yr), int(mt), int(d))
locB_a1_rain

,YEAR,MONTH,DAY,RAINFALL (mm),Date
0,1986,1,1,0.3,1986-01-01
1,1986,1,2,0,1986-01-02
2,1986,1,3,1.3,1986-01-03
3,1986,1,4,0.4,1986-01-04
4,1986,1,5,0,1986-01-05
...,...,...,...,...,...
13144,2021,12,27,0,2021-12-27
13145,2021,12,28,0,2021-12-28
13146,2021,12,29,0,2021-12-29
13147,2021,12,30,3,2021-12-30


In [24]:
locB_a1_temps.columns,locB_a1_rain.columns

(Index(['year', 'Month', 'Max_Temp', 'Min_Temp', 'Date', 'Avg_Temp', 'doy'], dtype='object'),
 Index(['YEAR', 'MONTH', 'DAY', 'RAINFALL (mm)', 'Date'], dtype='object'))

In [37]:
df = locB_a1_temps.merge(locB_a1_rain[["RAINFALL (mm)",'Date']], left_on='Date', right_on='Date', how='left')
df['year'] = df.year.astype(int)
df['Month'] = df.Month.astype(int)
df['area'] = 1
df['country'] = "B"
df = df.rename(columns={"RAINFALL (mm)":"rain"})[["doy","Month","year","rain","Date","Avg_Temp","Min_Temp","Max_Temp","area","country"]]
df['rain'] = df.rain.apply(lambda x: float(x) if x not in ['n.a.','**','T','0.0.', '  T', ' T', '0.T'] else np.nan)
df

,doy,Month,year,rain,Date,Avg_Temp,Min_Temp,Max_Temp,area,country
0,1,1,1986,0.3,1986-01-01,26.65,23.5,29.8,1,B
1,2,1,1986,0.0,1986-01-02,26.35,23.1,29.6,1,B
2,3,1,1986,1.3,1986-01-03,26.75,24.1,29.4,1,B
3,4,1,1986,0.4,1986-01-04,25.70,22.0,29.4,1,B
4,5,1,1986,0.0,1986-01-05,25.95,22.4,29.5,1,B
...,...,...,...,...,...,...,...,...,...,...
13119,361,12,2021,0.0,2021-12-27,27.30,24.0,30.6,1,B
13120,362,12,2021,0.0,2021-12-28,26.70,23.0,30.4,1,B
13121,363,12,2021,0.0,2021-12-29,27.65,24.4,30.9,1,B
13122,364,12,2021,3.0,2021-12-30,28.00,25.4,30.6,1,B


In [38]:
df.to_parquet("CountryB_Area1_cleaned_merged.parquet")